In [1]:
# import opensim as osim
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from OA_utils.data_utils import *
import random
import pickle

Load data from file

In [2]:
OA_data_dir = "C:\\Users\\bakel\\Desktop\\GRFMuscleModel\\Old_Young_Walking_Data\\"
with open(OA_data_dir + 'resampled_compiled_segments', 'rb') as f:
    segs = pickle.load(f)

In [5]:
for subj, data in segs.items():
    if subj == "time_resampled":
        continue  # skip singleton key
    
    # pick any signal key that exists in every subject
    num_segments = len(data["grf_y"])
    print(f"{subj}: {num_segments} segments")

OA1: 42 segments
OA2: 43 segments
OA4: 41 segments
OA5: 39 segments
OA7: 45 segments
OA8: 44 segments
OA9: 42 segments
OA10: 39 segments
OA11: 25 segments
OA12: 38 segments
OA13: 44 segments
OA14: 43 segments
OA17: 37 segments
OA18: 39 segments
OA19: 25 segments
OA20: 22 segments
OA22: 44 segments
OA23: 44 segments
OA24: 38 segments
OA25: 44 segments


In [3]:
OA_data_dir = "C:\\Users\\bakel\\Desktop\\GRFMuscleModel\\Old_Young_Walking_Data\\"
YA_data_dir = "C:\\Users\\bakel\\Desktop\\GRFMuscleModel\\data\\"
# load the data
OA_grf_data = np.load(OA_data_dir + 'OA_no_missteps_grf.npy', mmap_mode='r')
OA_muscle_data = np.load(OA_data_dir + 'OA_no_missteps_muscle.npy', mmap_mode='r')
YA_grf_data = np.load(YA_data_dir + 'grf.npy', mmap_mode='r')
YA_muscle_data = np.load(YA_data_dir + 'muscle.npy', mmap_mode='r')

grf_labels = ['GRF_x', 'GRF_y', 'GRF_z']
muscle_labels = ['tibpost', 'tibant', 'edl', 'ehl', 'fdl', 'fhl', 'perbrev', 'perlong', 'achilles']

grf_dict = {0: 'GRF_x', 1: 'GRF_y', 2: 'GRF_z'}
muscle_dict = {0: 'tibpost', 1: 'tibant', 2: 'edl', 3: 'ehl', 4: 'fdl', 5: 'fhl', 6: 'perbrev', 7: 'perlong', 8: 'achilles'}

In [ ]:
OA_data = np.concatenate((OA_grf_data, OA_muscle_data), axis=2)
YA_data = np.concatenate((YA_grf_data, YA_muscle_data), axis=2)
YA_resampled = YA_data[np.random.choice(len(YA_data), 766, replace=False)]
combined_data = np.concatenate([OA_data, YA_resampled], axis=0)
print("Combined shape:", combined_data.shape)

Combined shape: (1532, 100, 12)


In [ ]:
OA_data = np.concatenate((OA_grf_data, OA_muscle_data), axis=2)
# print(data.shape)

np.random.seed(42)
np.random.shuffle(data)

In [10]:
num_samples = combined_data.shape[0]
train_size = int(0.8 * num_samples)
val_size = int(0.1 * num_samples)

print(f"Number of samples: {num_samples}")
print(f"Train size: {train_size}")
print(f"Validation size: {val_size}")

Number of samples: 1532
Train size: 1225
Validation size: 153


In [14]:
train_data = combined_data[:train_size]
val_data = combined_data[train_size:train_size+val_size]
test_data = combined_data[train_size + val_size:]
print(f"Train data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

Train data shape: (1225, 100, 12)
Validation data shape: (153, 100, 12)
Test data shape: (154, 100, 12)


In [15]:
X_train, y_train = train_data[:, :, :3], train_data[:, :, 3:]
X_val, y_val = val_data[:, :, :3], val_data[:, :, 3:]
X_test, y_test = test_data[:, :, :3], test_data[:, :, 3:]

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1225, 100, 3)
y_train shape: (1225, 100, 9)
X_val shape: (153, 100, 3)
y_val shape: (153, 100, 9)
X_test shape: (154, 100, 3)
y_test shape: (154, 100, 9)


In [16]:
np.savez(OA_data_dir + 'mixed_train_data.npz', X_train=X_train, y_train=y_train)
np.savez(OA_data_dir + 'mixed_val_data.npz', X_val=X_val, y_val=y_val)
np.savez(OA_data_dir + 'mixed_test_data.npz', X_test=X_test, y_test=y_test)